In [133]:
import numpy as np
from heapq import heappush, heappop

设特征空间$\mathcal{X}$是$n$维实数向量空间$R^{n}$，$x_{i},x_{j} \in \mathcal{X},x_{i} = \left( x_{i}^{\left( 1 \right)},x_{i}^{\left( 2 \right) },\cdots,x_{i}^{\left( n \right) } \right)^{T},x_{j} = \left( x_{j}^{\left( 1 \right)},x_{j}^{\left( 2 \right) },\cdots,x_{j}^{\left( n \right) } \right)^{T}$，$x_{i},x_{j}$的$L_{p}$距离或Minkowski（闵科夫斯基）距离
\begin{align*}  \\ & L_{p} \left( x_{i},x_{j} \right) = \left( \sum_{l=1}^{N} \left| x_{i}^{\left(l\right)} - x_{j}^{\left( l \right)} \right|^{p} \right)^{\dfrac{1}{p}}\end{align*}  
其中，$p \geq 1$。当$p=2$时，称为欧氏距离，即
\begin{align*}  \\ & L_{2} \left( x_{i},x_{j} \right) = \left( \sum_{l=1}^{N} \left| x_{i}^{\left(l\right)} - x_{j}^{\left( l \right)} \right|^{2} \right)^{\dfrac{1}{2}}\end{align*}  
当$p=1$时，称为曼哈顿距离，即
\begin{align*}  \\ & L_{1} \left( x_{i},x_{j} \right) =  \sum_{l=1}^{N} \left| x_{i}^{\left(l\right)} - x_{j}^{\left( l \right)} \right| \end{align*} 
当$p=\infty$时，称为切比雪夫距离，是各个坐标距离的最大值，即
\begin{align*}  \\ & L_{\infty} \left( x_{i},x_{j} \right) =  \max_{l} \left| x_{i}^{\left(l\right)} - x_{j}^{\left( l \right)} \right| \end{align*} 

In [134]:
def minkowski_distance_p(x, y, p=2):
    """
    Parameters:
    ------------
    x: (M, K) array_like 
    y: (M, K) array_like
    p:  float, 1<= p <= infinity
    ------------
    计算M个K维向量的距离，但是该距离没有开p次根号
    """
    #把输入的array_like类型的数据转换成numpy中的ndarray
    x = np.asarray(x)
    y = np.asarray(y)
    ##axis=-1沿最后一个坐标轴，0，1沿着第一，第二个坐标轴
    if p == np.inf:
        return np.max(np.abs(x-y), axis=-1)
    else:
        return np.sum(np.abs(x-y)**p, axis=-1)
    
def minkowski_distance(x, y, p=2):
    if p==np.inf:
        return minkowski_distance_p(x, y, np.inf)
    else:
        return minkowski_distance_p(x, y, p)**(1./p)

平衡kd树构造算法：  
输入：$k$维空间数据集$T = \left\{  x_{1}, x_{2}, \cdots, x_{N} \right\}$，其中$x_{i} = \left( x_{i}^{\left(1\right)}, x_{i}^{\left(1\right)},\cdots,x_{i}^{\left(k\right)} \right)^{T}, i = 1, 2, \cdots, N$；  
输出：kd树  
1. 开始：构造根结点，根结点对应于包涵$T$的$k$维空间的超矩形区域。   
选择$x^{\left( 1 \right)}$为坐标轴，以$T$中所欲实例的$x^{\left( 1 \right)}$坐标的中位数为切分点，将根结点对应的超矩形区域切分成两个子区域。切分由通过切分点并与坐标轴$x^{\left( 1 \right)}$垂直的超平面实现。  
由根结点生成深度为1的左、右子结点：坐子结点对应坐标$x^{\left( 1 \right)}$小于切分点的子区域，右子结点对应于坐标$x^{\left( 1 \right)}$大与切分点的子区域。  
将落在切分超平面上的实例点保存在跟结点。
2. 重复：对深度为$j$的结点，选择$x^{\left( l \right)}$为切分坐标轴，$l = j \left(\bmod k \right) + 1 $，以该结点的区域中所由实例的$x^{\left( l \right)}$坐标的中位数为切分点，将该结点对应的超矩形区域切分为两个子区域。切分由通过切分点并与坐标轴$x^{\left( l \right)}$垂直的超平面实现。  
由根结点生成深度为$j+1$的左、右子结点：坐子结点对应坐标$x^{\left( l \right)}$小于切分点的子区域，右子结点对应于坐标$x^{\left( l \right)}$大与切分点的子区域。  
将落在切分超平面上的实例点保存在跟结点。
3. 直到两个子区域没有实例存在时停止。

关于kd树划分的几点说明：
1. 切分的粒度不用那么细，叶子结点上可以保留多个值，规定叶子结点的大小就好了。
2. 对划分坐标轴的选取可以采用维度最大间隔的方法,使用第d个坐标，该坐标点的最大值和最小值差最大。$argmax\{d\} = max\{max\{x^d\}-min\{x^d\}\}$
3. 使用平均值而不是采用中位数作为切分点

最后两点的结合以及一些细节在论文[Analysis of Approximate Nearest Neighbor
Searching with Clustered Point Sets](https://arxiv.org/abs/cs/9901013)中称为 Sliding-midpoint split。最后的实现也采用了这种方法，以及大量的参考了[Scipy中kd树实现代码](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.spatial.KDTree.html#scipy.spatial.KDTree)

In [135]:
"""
定义一个超矩形区域
"""
class Rectangle(object):
    """Hyperrectangle class.
    Represents a Cartesian product of intervals.
    """
    def __init__(self, maxes, mins):
        """Construct a hyperrectangle."""
        self.maxes = np.maximum(maxes,mins).astype(np.float)
        self.mins = np.minimum(maxes,mins).astype(np.float)
        self.m, = self.maxes.shape

    def __repr__(self):
        return "<Rectangle %s>" % list(zip(self.mins, self.maxes))

    def volume(self):
        """Total volume."""
        return np.prod(self.maxes-self.mins)

    def split(self, d, split):
        """
        Produce two hyperrectangles by splitting.
        In general, if you need to compute maximum and minimum
        distances to the children, it can be done more efficiently
        by updating the maximum and minimum distances to the parent.
        Parameters
        ----------
        d : int
            Axis to split hyperrectangle along.
        split :
            Input.
        """
        mid = np.copy(self.maxes)
        mid[d] = split
        less = Rectangle(self.mins, mid)
        mid = np.copy(self.mins)
        mid[d] = split
        greater = Rectangle(mid, self.maxes)
        return less, greater

kd树的最近邻搜索算法：  
输入：kd树；目标点$x$  
输出：$x$的最近邻  
1. 在kd树中找出包含目标点$x$的叶结点：从跟结点出发，递归地向下访问kd树。若目标点$x$当前维的坐标小于切分点的坐标，则移动到左子结点，否则移动到右子结点。直到子结点为叶结点为止。   
2. 以此叶结点为“当前最近点”。
3. 递归地向上回退，在每个结点进行以下操作：  
3.1 如果该结点保存的实例点比当前最近点距离目标点更近，则以该实例点为“当前最近点”。  
3.2 当前最近点一定存在于该结点一个子结点对应的区域。检查该子结点的父结点的另一子结点对应的区域是否有更近的点。具体地，检查另一子结点对应的区域是否与以目标点为球心、以目标点与“当前最近点”间的距离为半径的超球体相交。  
如果相交，可能在另一个子结点对应的区域内存在距目标点更近的点，移动到另一个子结点。接着，递归地进行最近邻搜索；  
如果不相交，向上回退。  
4. 当回退到根结点时，搜索结束。最后的“当前最近点”即为$x$的当前最近邻点。

kd树的k近邻搜索算法，需要使用优先队列$neighbors$保存$k$个搜索结果，搜索的过程也可以不使用递归，用另外一个优先队列$q$来存储接下来需要搜索的结点

In [148]:
class KDTree(object):
    def __init__(self, data, leafsize=10):
        """
        
        """
        self.data= np.asarray(data)
        self.n, self.m = self.data.shape
        self.leafsize = leafsize
        if self.leafsize < 1:
            raise ValueError("leafsize must be at least 1")
        self.maxes = np.max(self.data, axis=0)
        self.mins = np.min(self.data, axis=0)
        self.tree = self.__build(np.arange(self.n), self.maxes, self.mins)
    
    """
    定义结点类，作为叶子结点和内部结点的父类
    是必须重写比较运算符吗？？？
    """
    class node(object):
        def __it__(self, other):
            return id(self) < id(other)
        def __gt__(self, other):
            return id(self) > id(other)
        def __le__(self, other):
            return id(self) <= id(other)
        def __ge__(self, other):
            return id(self) >= id(other)
        def __eq__(self, other):
            return id(self) == id(other)
        
    """
    定义叶子结点
    """
    class leafnode(node):
        def __init__(self, idx):
            self.idx = idx
            self.children = len(self.idx)
    """
    定义内部结点
    """
    class innernode(node):
        def __init__(self, split_dim, split, less, greater):
            """
            split_dim: 在某个维度上进行的划分
            split：在该维度上的划分点
            """
            self.split_dim = split_dim
            self.split = split
            self.less = less
            self.greater = greater
            self.children = less.children+greater.children
            
    """
    仅开头带双下划线__的命名 用于对象的数据封装，以此命名的属性或者方法为类的私有属性或者私有方法。
    如果在外部直接访问私有属性或者方法,是不可行的，这就起到了隐藏数据的作用。
    但是这种实现机制并不是很严格，机制是通过自动"变形"实现的，类中所有以双下划线开头的名称__name都会自动变为"_类名__name"的新名称。
    使用"_类名__name"就可以访问了，如._KDTree__build()。同时机制可以阻止继承类重新定义或者更改方法的实现。
    
    """
    def __build(self, idx, maxes, mins):
        if len(idx) <= self.leafsize:
            return KDTree.leafnode(idx)
        else:
            #在第d维上进行划分，选自第d维的依据是该维度的间隔最大
            d = np.argmax(maxes-mins)
            #第d维上的数据
            data = self.data[idx][d]
            #第d维上的区间端点
            maxval, minval = maxes[d], mins[d]
        if maxval == minval:
            #所有的点值都相同
            return KDTree.leafnode(idx)
        """
         Splitting Methods
        sliding midpoint rule;
        see Maneewongvatana and Mount 1999"""
        split = (maxval + minval) / 2
        #分别返回小于等于，大于split值的元素的索引
        less_idx = np.nonzero(data <= split)[0] 
        greater_idx = np.nonzero(data>split)[0]
        #对于极端的划分情况进行调整
        if len(less_idx) == 0:
                split = np.min(data)
                less_idx = np.nonzero(data <= split)[0]
                greater_idx = np.nonzero(data > split)[0]
        if len(greater_idx) == 0:
                split = np.max(data)
                less_idx = np.nonzero(data < split)[0]
                greater_idx = np.nonzero(data >= split)[0]
        if len(less_idx) == 0:
                # _still_ zero? all must have the same value
            if not np.all(data == data[0]):
                raise ValueError("Troublesome data array: %s" % data)
            split = data[0]
            less_idx = np.arange(len(data)-1)
            greater_idx = np.array([len(data)-1])
        #递归调用左边和右边
        lessmaxes = np.copy(maxes)
        lessmaxes[d] = split
        greatermins = np.copy(mins)
        greatermins[d] = split
        return KDTree.innernode(d, split,
                    self.__build(idx[less_idx],lessmaxes,mins),
                    self.__build(idx[greater_idx],maxes,greatermins))
    
    def query(self, x, k=1, p=2, distance_upper_bound=np.inf):
        x = np.asarray(x)
        #距离下界，形象化的思考一下哈，点x出现在mins和maxes的位置分三种情况
        side_distances = np.maximum(0,np.maximum(x-self.maxes,self.mins-x))
        if p != np.inf:
            side_distances **= p
            min_distance = np.sum(side_distances)
        else:
            min_distance = np.amax(side_distances)
            
        if p != np.inf and distance_upper_bound != np.inf:
            distance_upper_bound = distance_upper_bound**p
    
        q, neighbors = [(min_distance, tuple(side_distances), self.tree)], []
        """
        q: 维护搜索的优先队列 
        # entries are:
        (minimum distance between the cell and the target, 
        distances between the nearest side of the cell and the target, 
        the head node of the cell)
        neighbors: priority queue for the nearest neighbors
        用于保存k近邻结果的优先队列，heapq默认是最小堆，为了立即能够得到队列中点的最大距离来更新距离上界upper bound，可以存储距离的相反数。
        #entries are (-distance**p, index)
        
        """

        while q:
            min_distance, side_distances, node = heappop(q)
            if isinstance(node, KDTree.leafnode):
                # 对于叶子结点，就一个个暴力排除
                data = self.data[node.idx]
                # 把x沿x-轴扩充，然后和叶子结点上的点比较大小
                ds = minkowski_distance_p(data,x[np.newaxis,:],p)
                
                for i in range(len(ds)):
                    if ds[i] < distance_upper_bound:
                        if len(neighbors) == k:
                            heappop(neighbors)
                        heappush(neighbors, (-ds[i], node.idx[i]))
                        if len(neighbors) == k: #更新距离上界
                            distance_upper_bound = -neighbors[0][0]
            else:
                # we don't push cells that are too far onto the queue at all,
                # but since the distance_upper_bound decreases, we might get
                # here even if the cell's too far
                if min_distance > distance_upper_bound:
                    # since this is the nearest cell, we're done, bail out
                    break
                # compute minimum distances to the children and push them on
                if x[node.split_dim] < node.split:
                    near, far = node.less, node.greater
                else:
                    near, far = node.greater, node.less
                # near child is at the same distance as the current node
                heappush(q,(min_distance, side_distances, near))
                
                # 对于far child需要进行距离判断，用新的距离替代原来的距离，然后和距离上界比较 
                sd = list(side_distances)
                if p == np.inf:
                    min_distance = max(min_distance, abs(node.split-x[node.split_dim]))
                elif p == 1:
                    sd[node.split_dim] = np.abs(node.split-x[node.split_dim])
                    min_distance = (min_distance - side_distances[node.split_dim]) + sd[node.split_dim]
                else:
                    sd[node.split_dim] = np.abs(node.split-x[node.split_dim])**p
                    min_distance = (min_distance - side_distances[node.split_dim]) + sd[node.split_dim]

                if min_distance <= distance_upper_bound:
                    heappush(q,(min_distance, tuple(sd), far))

        if p == np.inf:
            return sorted([(-d,i) for (d,i) in neighbors])
        else:
            return sorted([((-d)**(1./p),i) for (d,i) in neighbors])

In [140]:
data = [[2,3],[5,4],[9,6],[4,7],[8,1],[7,2]]
kd = KDTree(data)

In [141]:
ans = kd.query([2, 1], k=2)
for item in ans:
    print("距离：%f， 索引:%d" %(item[0], item[1]))

距离：2.000000， 索引:0
距离：4.242641， 索引:1


In [146]:
np.random.random((2,3))

array([[0.52748925, 0.10629922, 0.92090696],
       [0.93105817, 0.15561812, 0.2538185 ]])

In [150]:
from time import clock
t0 = clock()
kd2 = KDTree(np.random.random(( 400000, 3)))            # 构建包含四十万个3维空间样本点的kd树
ret2 =  kd2.query([0.1,0.5,0.8])      # 四十万个样本点中寻找离目标最近的点
t1 = clock()
print ("time: ",t1-t0, "s")
print (ret2)

time:  0.08968533333330697 s
[(0.23022348089627068, 1)]
